In [1]:
import torch
from torch import nn
import pandas as pd
from torch.utils.data import TensorDataset,Dataset,DataLoader,random_split

def normalize(dataset):
    dataNorm=((dataset-dataset.min())/(dataset.max()-dataset.min()))
    dataNorm["CreditLevel"]=dataset["CreditLevel"]-1
    return dataNorm

# data preprocessing
train_dataset = pd.read_csv("BankChurners.csv")
print(train_dataset.shape)
#train_dataset.head(10)
train_dataset = train_dataset.drop(['CustomerId','Geography'],axis=1)
train_dataset = normalize(train_dataset)
print(train_dataset.shape)
train_dataset.head()


(9000, 10)
(9000, 8)


,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,CreditLevel
0,0.3,0.484985,0.000000,1.0,0.0,0.643290,1.0,7
1,0.6,0.000000,0.000000,1.0,0.0,0.251062,1.0,6
2,0.2,0.728934,0.000000,1.0,0.0,0.015250,0.0,6
3,0.2,0.407651,0.333333,1.0,0.0,0.449146,0.0,1
4,0.7,0.435819,0.333333,1.0,0.0,0.513377,0.0,6


In [2]:
x_train = torch.tensor(train_dataset.drop('CreditLevel',axis=1).values.astype('float32'))
y_train = torch.tensor(train_dataset['CreditLevel'].values)
train_tensor = TensorDataset(x_train,y_train)
n_train = int(len(train_tensor)*0.8)
n_valid = len(train_tensor) - n_train
ds_train,ds_valid = random_split(train_tensor,[n_train,n_valid])
dl_train = DataLoader(ds_train,batch_size = 8,shuffle=True)
dl_valid = DataLoader(ds_valid,batch_size = 8,shuffle=False)

for features,labels in dl_train:
    print(features,labels)
    break

tensor([[0.2000, 0.5443, 0.3333, 1.0000, 1.0000, 0.6468, 0.0000],
        [0.5000, 0.0000, 0.3333, 1.0000, 0.0000, 0.7950, 0.0000],
        [0.6000, 0.5571, 0.0000, 1.0000, 0.0000, 0.1372, 1.0000],
        [0.3000, 0.3923, 0.0000, 0.0000, 0.0000, 0.3785, 0.0000],
        [0.1000, 0.6503, 0.0000, 0.0000, 1.0000, 0.0780, 0.0000],
        [0.5000, 0.5596, 0.3333, 0.0000, 1.0000, 0.2858, 0.0000],
        [0.7000, 0.5336, 0.0000, 1.0000, 1.0000, 0.1364, 0.0000],
        [0.7000, 0.6567, 0.6667, 1.0000, 0.0000, 0.5991, 1.0000]]) tensor([3, 6, 6, 2, 6, 6, 2, 4])


In [3]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.init import kaiming_uniform_,xavier_uniform_

class Net(nn.Module):
    def __init__(self,n_inputs):
        super(Net, self).__init__()
        # input to first hidden layer
        self.hidden1 = nn.Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = nn.ReLU()
        # second hidden layer
        self.hidden2 = nn.Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = nn.ReLU()
        # third hidden layer and output
        self.hidden3 = nn.Linear(8, 10)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = nn.Softmax(dim=1)
    
    def forward(self,X):
         # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
        # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # output layer
        X = self.hidden3(X)
        X = self.act3(X)
        return X

model = Net(7)
print(model)

Net(
  (hidden1): Linear(in_features=7, out_features=10, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=10, out_features=8, bias=True)
  (act2): ReLU()
  (hidden3): Linear(in_features=8, out_features=10, bias=True)
  (act3): Softmax(dim=1)
)


In [4]:
#define a Loss function and optimizer
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [5]:
# Train the network
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs,targets) in enumerate(dl_train):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,targets)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 100 == 0:   
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

[1,     1] loss: 0.023
[1,   101] loss: 2.309
[1,   201] loss: 2.305
[1,   301] loss: 2.294
[1,   401] loss: 2.292
[1,   501] loss: 2.286
[1,   601] loss: 2.276
[1,   701] loss: 2.275
[1,   801] loss: 2.264
[2,     1] loss: 0.022
[2,   101] loss: 2.253
[2,   201] loss: 2.257
[2,   301] loss: 2.255
[2,   401] loss: 2.246
[2,   501] loss: 2.252
[2,   601] loss: 2.231
[2,   701] loss: 2.237
[2,   801] loss: 2.252
[3,     1] loss: 0.021
[3,   101] loss: 2.230
[3,   201] loss: 2.226
[3,   301] loss: 2.225
[3,   401] loss: 2.232
[3,   501] loss: 2.225
[3,   601] loss: 2.241
[3,   701] loss: 2.224
[3,   801] loss: 2.211
[4,     1] loss: 0.022
[4,   101] loss: 2.199
[4,   201] loss: 2.226
[4,   301] loss: 2.220
[4,   401] loss: 2.216
[4,   501] loss: 2.226
[4,   601] loss: 2.218
[4,   701] loss: 2.223
[4,   801] loss: 2.226
[5,     1] loss: 0.022
[5,   101] loss: 2.221
[5,   201] loss: 2.219
[5,   301] loss: 2.218
[5,   401] loss: 2.216
[5,   501] loss: 2.220
[5,   601] loss: 2.219
[5,   701] 

In [8]:
# model test
model.eval()
accuracy = 0.0
total = 0.0
with torch.no_grad():
    for data in dl_valid:
        inputs,targets = data
        outputs = model(inputs)
        _,predicted = torch.max(outputs.data,1)
        total += targets.size(0)
        accuracy += (predicted == targets).sum().item()

accuracy = (100*accuracy/total)
accuracy

20.77777777777778